Transformation is done based on this document -- [Link](https://docs.google.com/document/d/1_wt-wO18sa5iQBy3b-WpSHDjCqRl3MeN67N1v3dTZtQ/edit#heading=h.usgit3agfa62)

In [1]:
import pandas as pd
from shapely.geometry import Point
import geopandas as gpd
from geopandas import GeoDataFrame

In [2]:
# importing village shapefile from bharatmaps - 
bharat_maps_villages = gpd.read_file(r'D:\CivicDataLabs_IDS-DRR\IDS-DRR_Github\flood-data-ecosystem-Odisha\Maps\Geojson\bm_odisha_villages_filtered_reduced.geojson')

Can't load requested DLL: C:\OSGeo4W\apps\gdal\lib\gdalplugins\gdal_ECW_JP2ECW.dll
127: The specified procedure could not be found.

Can't load requested DLL: C:\OSGeo4W\apps\gdal\lib\gdalplugins\gdal_ECW_JP2ECW.dll
127: The specified procedure could not be found.

Can't load requested DLL: C:\OSGeo4W\apps\gdal\lib\gdalplugins\gdal_GEOR.dll
127: The specified procedure could not be found.

Can't load requested DLL: C:\OSGeo4W\apps\gdal\lib\gdalplugins\gdal_GEOR.dll
127: The specified procedure could not be found.

Can't load requested DLL: C:\OSGeo4W\apps\gdal\lib\gdalplugins\gdal_HDF5.dll
127: The specified procedure could not be found.

Can't load requested DLL: C:\OSGeo4W\apps\gdal\lib\gdalplugins\gdal_HDF5.dll
127: The specified procedure could not be found.

Can't load requested DLL: C:\OSGeo4W\apps\gdal\lib\gdalplugins\gdal_MrSID.dll
127: The specified procedure could not be found.

Can't load requested DLL: C:\OSGeo4W\apps\gdal\lib\gdalplugins\gdal_MrSID.dll
127: The specified p

In [3]:
%pip install openpyxl

Note: you may need to restart the kernel to use updated packages.


In [4]:
mission_antyodaya_df = pd.read_excel(r'D:\CivicDataLabs_IDS-DRR\IDS-DRR_Github\flood-data-ecosystem-Odisha\Sources\ANTYODAYA\Odisha_antyodaya_village_dataset.xlsx')

In [5]:
mission_antyodaya_df.shape

(49535, 202)

In [6]:
mission_antyodaya_geometry = [
        Point(xy) for xy in zip(
            mission_antyodaya_df.village_longitude,
            mission_antyodaya_df.village_latitude
        )
    ]

In [7]:
gdf_points = GeoDataFrame(mission_antyodaya_df, geometry=mission_antyodaya_geometry)

In [8]:
# importing odisha subdistrict shapefile as proxy for revenue circles
gdf_polygons = gpd.read_file(r'D:\CivicDataLabs_IDS-DRR\IDS-DRR_Github\flood-data-ecosystem-Odisha\Maps\Geojson\bharatmaps_odisha_subdistrict.geojson')
#gdf_polygons = gdf_polygons.rename(columns = {'SDTCODE11':'sub_district_code'}, inplace = True)

In [9]:
gdf_points.crs = gdf_polygons.crs

In [10]:
result = gpd.sjoin(gdf_points, gdf_polygons, how="left", op="within")

d:\CivicDataLabs_IDS-DRR\IDS-DRR_Github\IDS-DRR-Assam\.venv\Lib\site-packages\IPython\core\interactiveshell.py:3517: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):


In [11]:
result.tail()

,state_code,state_name,district_code,district_name,sub_district_code,sub_district_name,block_code,block_name,gp_code,gp_name,...,dtcode11,sdtcode11,Shape_Length,Shape_Area,stname,dtname,sdtname,Subdt_LGD,Dist_LGD,State_LGD
49530,21,ODISHA,350,CUTTACK,2946,Badamba,3333,BADAMBA,116877,GOPAMATHURA,...,381,02946,102031.183065,4.209133e+08,ODISHA,Cuttack,Badamba,2946.0,350.0,21.0
49531,21,ODISHA,359,KANDHAMAL,3121,Tumudibandha,3428,KOTAGARH,118953,PARIGARH,...,390,03119,290666.251003,1.273240e+09,ODISHA,Kandhamal,Baliguda,3119.0,359.0,21.0
49532,21,ODISHA,359,KANDHAMAL,3123,Kotagarh,3428,KOTAGARH,118953,PARIGARH,...,390,03119,290666.251003,1.273240e+09,ODISHA,Kandhamal,Baliguda,3119.0,359.0,21.0
49533,21,ODISHA,359,KANDHAMAL,3123,Kotagarh,3428,KOTAGARH,118953,PARIGARH,...,390,03119,290666.251003,1.273240e+09,ODISHA,Kandhamal,Baliguda,3119.0,359.0,21.0
49534,21,ODISHA,359,KANDHAMAL,3123,Kotagarh,3428,KOTAGARH,118953,PARIGARH,...,390,03119,290666.251003,1.273240e+09,ODISHA,Kandhamal,Baliguda,3119.0,359.0,21.0


In [12]:
#renaming sub-district code columns to match
gdf_polygons_renamed = gdf_polygons.rename(columns = {'sdtcode11':'sub_district_code'})

In [13]:
#gdf_polygons and bharat_maps_villages have slightly different sub-district codes. removing unnecessary trailing'0'
gdf_polygons_renamed['sub_district_code'] = gdf_polygons_renamed['sub_district_code'].apply(lambda x: x[1:] if x.startswith('0') else x)
gdf_polygons_renamed

,OBJECTID,stcode11,dtcode11,sub_district_code,Shape_Length,Shape_Area,stname,dtname,sdtname,Subdt_LGD,Dist_LGD,State_LGD,geometry
0,2773,21,370,2755,175809.421683,6.516630e+08,ODISHA,Bargarh,Paikamal,2755,347,21,"POLYGON ((82.90633 21.04445, 82.90599 21.04534..."
1,2774,21,370,2756,192057.189614,7.626096e+08,ODISHA,Bargarh,Jharbandha,2756,347,21,"POLYGON ((82.96518 21.19248, 82.96504 21.19318..."
2,2775,21,370,2757,133403.031651,3.456675e+08,ODISHA,Bargarh,Padmapur,2757,347,21,"POLYGON ((83.16820 21.13856, 83.16775 21.13887..."
3,2776,21,370,2758,113949.812620,2.672039e+08,ODISHA,Bargarh,BurdenP.S.,2758,347,21,"POLYGON ((83.12346 20.98452, 83.12192 20.98650..."
4,2777,21,370,2759,135841.048349,3.752092e+08,ODISHA,Bargarh,Gaisilet,2759,347,21,"POLYGON ((83.28376 21.04885, 83.28268 21.04889..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
473,3246,21,399,3226,149569.992418,5.100200e+08,ODISHA,Malkangiri,Orkel,3226,364,21,"POLYGON ((82.16914 18.48265, 82.16793 18.48307..."
474,3247,21,399,3227,269711.225643,9.885963e+08,ODISHA,Malkangiri,Kalimela,3227,364,21,"POLYGON ((81.72320 18.29129, 81.72195 18.29135..."
475,3248,21,399,3228,129218.450510,4.291734e+08,ODISHA,Malkangiri,Podia,3228,364,21,"POLYGON ((81.66331 18.32195, 81.66271 18.32223..."
476,3249,21,399,3229,130042.312962,3.676498e+08,ODISHA,Malkangiri,M.V. 79,3229,364,21,"POLYGON ((81.64158 18.10071, 81.64075 18.10174..."


In [14]:
#The splitting into tagged+untagged has been skipped as all the points seem to have an associated subdistrict - leaving it in in case this needs to be changed later
#untagged = result[result.sub_district_code.isnull()]
#tagged = result.dropna(subset=['sub_district_code'])

In [15]:
#Original code for revenue circles - not removing as it may be required later
#untagged_bm = bharat_maps_villages[bharat_maps_villages.vil_lgd.isin(untagged.village_code.to_list())]
#untagged_bm['geometry'] = untagged_bm.geometry.centroid
#untagged_bm = untagged_bm.to_crs('EPSG:4326')
#untagged_bm = gpd.sjoin(untagged_bm, gdf_polygons, how="left", op="within")

In [16]:
'''result_bm = bharat_maps_villages[bharat_maps_villages.vil_lgd.isin(result.village_code.to_list())]
result_bm['geometry'] = result_bm.geometry.centroid
result_bm = result_bm.to_crs('EPSG:4326')
result_bm = gpd.sjoin(result_bm, gdf_polygons, how="left", op="within")'''

'result_bm = bharat_maps_villages[bharat_maps_villages.vil_lgd.isin(result.village_code.to_list())]\nresult_bm[\'geometry\'] = result_bm.geometry.centroid\nresult_bm = result_bm.to_crs(\'EPSG:4326\')\nresult_bm = gpd.sjoin(result_bm, gdf_polygons, how="left", op="within")'

In [17]:
'''
untagged = pd.merge(untagged.drop(['object_id','revenue_ci'],axis=1),untagged_bm[['vil_lgd','object_id','revenue_ci']],
        left_on='village_code', right_on='vil_lgd')[untagged.columns]
        '''

"\nuntagged = pd.merge(untagged.drop(['object_id','revenue_ci'],axis=1),untagged_bm[['vil_lgd','object_id','revenue_ci']],\n        left_on='village_code', right_on='vil_lgd')[untagged.columns]\n        "

In [18]:
#result = pd.concat([tagged,untagged])

In [19]:
#result.to_csv(r'D:\CivicDataLabs_IDS-DRR\IDS-DRR_Github\flood-data-ecosystem-Odisha\Sources\ANTYODAYA\data/MissionAntyodaya2020_Odisha_untaggedRC.csv')

In [20]:
result[['gp_code','village_code','sub_district_code','sdtname','net_sown_area_in_hac','total_hhd',
        'availablility_hours_of_domestic_electricity', 'availability_of_telephone_services',
       'total_hhd_having_piped_water_connection', 'total_hhd_not_having_sanitary_latrines']].to_csv(
    r'D:\CivicDataLabs_IDS-DRR\IDS-DRR_Github\flood-data-ecosystem-Odisha\Sources\ANTYODAYA\data\MissionAntyodaya2020_Odisha_vul.csv', index=False)

In [21]:
#original with revenue circle
''' result[['gp_code','village_code','object_id','revenue_ci','net_sown_area_in_hac','total_hhd',
        'availablility_hours_of_domestic_electricity', 'availability_of_telephone_services',
       'total_hhd_having_piped_water_connection', 'total_hhd_not_having_sanitary_latrines']].to_csv(
    r'D:\CivicDataLabs_IDS-DRR\IDS-DRR_Github\flood-data-ecosystem-Odisha\Sources\ANTYODAYA\data\MissionAntyodaya2020_Odisha_vul.csv', index=False)'''

<>:2: SyntaxWarning: invalid escape sequence '\C'
<>:2: SyntaxWarning: invalid escape sequence '\C'
C:\Users\saura\AppData\Local\Temp\ipykernel_6916\1810735516.py:2: SyntaxWarning: invalid escape sequence '\C'
  ''' result[['gp_code','village_code','object_id','revenue_ci','net_sown_area_in_hac','total_hhd',


" result[['gp_code','village_code','object_id','revenue_ci','net_sown_area_in_hac','total_hhd',\n        'availablility_hours_of_domestic_electricity', 'availability_of_telephone_services',\n       'total_hhd_having_piped_water_connection', 'total_hhd_not_having_sanitary_latrines']].to_csv(\n    r'D:\\CivicDataLabs_IDS-DRR\\IDS-DRR_Github\x0clood-data-ecosystem-Odisha\\Sources\\ANTYODAYA\\data\\MissionAntyodaya2020_Odisha_vul.csv', index=False)"

# Number of households in each urban area

In [22]:
od_urban_shapes = gpd.read_file(r'D:\CivicDataLabs_IDS-DRR\IDS-DRR_Github\flood-data-ecosystem-Odisha\Maps\Geojson\od_urban_shapes.geojson')
od_urban_shapes

,VILNAM_SOI,DTNAME,STCODE11,DTCODE11,SDTCODE11,VILCODE11,SDTNAME,VILNAME11,geometry
0,KALLERU,EAST GODAVARI,37,545,04732,579203,Chintur,Kalleru,"MULTIPOLYGON (((9060973.271 2016489.652, 90609..."
1,DEVARAPALLI R F,EAST GODAVARI,37,545,04732,579218,Chintur,Kesaram,"MULTIPOLYGON (((9072140.588 2015674.695, 90719..."
2,Nelakota,EAST GODAVARI,37,545,04732,579210,Chintur,Nelakota,"MULTIPOLYGON (((9082259.849 2018432.849, 90821..."
3,MADUGUR,EAST GODAVARI,37,545,04732,579204,Chintur,Madugur,"MULTIPOLYGON (((9066453.893 2017135.331, 90662..."
4,Vemularayi,EAST GODAVARI,37,545,04732,579209,Chintur,Vemulrai,"MULTIPOLYGON (((9079685.357 2017371.805, 90796..."
...,...,...,...,...,...,...,...,...,...
2266,MUNDUI,PASHCHIMI SINGHBHUM,20,368,02743,378294,Jagannathpur,Mundui,"MULTIPOLYGON (((9543115.118 2519295.447, 95432..."
2267,JATNI TOWN,KHORDHA,21,386,03049,801856,Jatani,Jatani (M + OG),"MULTIPOLYGON (((9539863.910 2293150.772, 95401..."
2268,GADAJAGASORA,KHORDHA,21,386,03049,407791,Jatani,Gadajagasora,"MULTIPOLYGON (((9542383.998 2300776.351, 95423..."
2269,NARSINHPUR PATANA,PURI,21,387,03066,409877,Sadar,Narasinghpurpattna,"MULTIPOLYGON (((9541834.044 2248153.937, 95417..."


In [23]:
od_urban_shapes = od_urban_shapes[~(od_urban_shapes['geometry'].is_empty | od_urban_shapes['geometry'].isna())]


In [24]:
# TOTAL POPULATION IN EACH URBAN AREA
import rasterio
import rasterstats

worldpop_raster = rasterio.open(r'D:\CivicDataLabs_IDS-DRR\IDS-DRR_Github\flood-data-ecosystem-Odisha\Sources\WORLDPOP\odisha_ppp_2020_UNadj.tif')
worldpop_raster_array = worldpop_raster.read(1)

sum_dicts = rasterstats.zonal_stats(od_urban_shapes.to_crs(worldpop_raster.crs),
                                     worldpop_raster_array,
                                     affine= worldpop_raster.transform,
                                     stats= ['sum'],
                                     nodata=worldpop_raster.nodata,
                                     geojson_out = True)

dfs = []
for sd in sum_dicts:
    dfs.append(pd.DataFrame([sd['properties']]))

pop_zonal_stats_df = pd.concat(dfs).reset_index(drop=True)
pop_zonal_stats_df = pop_zonal_stats_df.rename(columns={'sum':'sum_population'})

In [ ]:
# Divide population by 4.3 for households
pop_zonal_stats_df['urban_hhd'] = pop_zonal_stats_df['sum_population']/4.3

In [ ]:
urban_hhd = pop_zonal_stats_df.groupby('SDTCODE11')[['urban_hhd']].sum().reset_index()
urban_hhd

,SDTCODE11,urban_hhd
0,02458,0.0
1,02467,0.0
2,02469,0.0
3,02491,0.0
4,02493,0.0
...,...,...
340,04846,0.0
341,04848,0.0
342,04849,0.0
343,04850,0.0


In [ ]:
urban_hhd['urban_electricity'] = urban_hhd['urban_hhd']*20

In [ ]:
urban_hhd['urban_tele'] = urban_hhd['urban_hhd']*3

In [ ]:
urban_hhd['urban_hhd_pipe'] = urban_hhd['urban_hhd']*0.1873

In [ ]:
urban_hhd['urban_no_sanitation'] = urban_hhd['urban_hhd']*0.063

In [ ]:
urban_hhd = urban_hhd.rename(columns={'SDTCODE11': 'sub_district_code'})

In [ ]:
urban_hhd['sub_district_code'] = urban_hhd['sub_district_code'].apply(lambda x: x[1:] if x.startswith('0') else x)
urban_hhd

,sub_district_code,urban_hhd,urban_electricity,urban_tele,urban_hhd_pipe,urban_no_sanitation
0,2458,0.0,0.0,0.0,0.0,0.0
1,2467,0.0,0.0,0.0,0.0,0.0
2,2469,0.0,0.0,0.0,0.0,0.0
3,2491,0.0,0.0,0.0,0.0,0.0
4,2493,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...
340,4846,0.0,0.0,0.0,0.0,0.0
341,4848,0.0,0.0,0.0,0.0,0.0
342,4849,0.0,0.0,0.0,0.0,0.0
343,4850,0.0,0.0,0.0,0.0,0.0


# Vul variables

In [ ]:
vul_df = pd.read_csv(r'D:\CivicDataLabs_IDS-DRR\IDS-DRR_Github\flood-data-ecosystem-Odisha\Sources\ANTYODAYA\data\MissionAntyodaya2020_Odisha_vul.csv')

In [ ]:
vul_df = vul_df.rename(columns={'total_hhd':'rural_hhd'})
vul_df['sub_district_code'] = vul_df['sub_district_code'].astype(str)


## net_sown_area_in_hac

In [ ]:
#original 
net_sown_area_in_hac_sd = vul_df.groupby(['sub_district_code'])[['net_sown_area_in_hac']].sum()

In [ ]:
net_sown_area_in_hac_sd = vul_df.groupby(['sub_district_code'])[['net_sown_area_in_hac']].sum()

In [ ]:
net_sown_area_in_hac_sd

,net_sown_area_in_hac
sub_district_code,
2755,20637.310
2756,24144.640
2757,14361.820
2758,8136.000
2759,11586.370
...,...
3226,9017.210
3227,7170.903
3228,7906.389


## Rural - Urban Households in each rc

In [ ]:
rural_hhd = vul_df.groupby('sub_district_code')[['rural_hhd']].sum().reset_index()
rural_hhd

,sub_district_code,rural_hhd
0,2755,23627
1,2756,31699
2,2757,17150
3,2758,8776
4,2759,21890
...,...,...
460,3226,14580
461,3227,31236
462,3228,6899
463,3229,14013


In [ ]:
vul_df[vul_df.sub_district_code == 2755]

,gp_code,village_code,sub_district_code,sdtname,net_sown_area_in_hac,rural_hhd,availablility_hours_of_domestic_electricity,availability_of_telephone_services,total_hhd_having_piped_water_connection,total_hhd_not_having_sanitary_latrines


In [ ]:
#urban_hhd['sub_district_code'] = urban_hhd['sub_district_code'].astype(str)
total_hhd = urban_hhd.merge(rural_hhd, how='outer', on='sub_district_code')
total_hhd

,sub_district_code,urban_hhd,urban_electricity,urban_tele,urban_hhd_pipe,urban_no_sanitation,rural_hhd
0,2458,0.0,0.0,0.0,0.0,0.0,NaN
1,2467,0.0,0.0,0.0,0.0,0.0,NaN
2,2469,0.0,0.0,0.0,0.0,0.0,NaN
3,2491,0.0,0.0,0.0,0.0,0.0,NaN
4,2493,0.0,0.0,0.0,0.0,0.0,NaN
...,...,...,...,...,...,...,...
545,4846,0.0,0.0,0.0,0.0,0.0,NaN
546,4848,0.0,0.0,0.0,0.0,0.0,NaN
547,4849,0.0,0.0,0.0,0.0,0.0,NaN
548,4850,0.0,0.0,0.0,0.0,0.0,NaN


In [ ]:
total_hhd = total_hhd.fillna(0)
total_hhd['total_hhd']=total_hhd['rural_hhd']+total_hhd['urban_hhd']

In [ ]:
total_hhd

,sub_district_code,urban_hhd,urban_electricity,urban_tele,urban_hhd_pipe,urban_no_sanitation,rural_hhd,total_hhd
0,2458,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2467,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2469,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2491,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2493,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...
545,4846,0.0,0.0,0.0,0.0,0.0,0.0,0.0
546,4848,0.0,0.0,0.0,0.0,0.0,0.0,0.0
547,4849,0.0,0.0,0.0,0.0,0.0,0.0,0.0
548,4850,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## availablility_hours_of_domestic_electricity

In [ ]:
# Map category to mean electricity available
electricity_dict = {1:2.5, 2:6, 3:10, 4:12, 5:0}

In [ ]:
elect_df = vul_df.replace({"availablility_hours_of_domestic_electricity": electricity_dict})[['sub_district_code','rural_hhd','availablility_hours_of_domestic_electricity']]

In [ ]:
elect_df['rural_electricity'] = elect_df['rural_hhd']*elect_df['availablility_hours_of_domestic_electricity']

In [ ]:
elect_df = elect_df.groupby('sub_district_code')[['rural_hhd','rural_electricity']].sum().reset_index()

In [ ]:
elect_df

,sub_district_code,rural_hhd,rural_electricity
0,2755,23627,203468.5
1,2756,31699,307670.0
2,2757,17150,169277.5
3,2758,8776,97482.0
4,2759,21890,207794.0
...,...,...,...
460,3226,14580,161810.0
461,3227,31236,279538.0
462,3228,6899,68571.5
463,3229,14013,143652.0


In [ ]:
elect_df['sub_district_code'] = elect_df['sub_district_code'].astype(str)
sd_electricity = elect_df.merge(urban_hhd, on='sub_district_code', how='outer')

In [ ]:
sd_electricity = sd_electricity.fillna(0)

In [ ]:
sd_electricity['total_electricity'] = sd_electricity['rural_electricity'] + sd_electricity['urban_electricity']

In [ ]:
sd_electricity['total_hhd'] = sd_electricity['rural_hhd'] + sd_electricity['urban_hhd']

In [ ]:
sd_electricity

,sub_district_code,rural_hhd,rural_electricity,urban_hhd,urban_electricity,urban_tele,urban_hhd_pipe,urban_no_sanitation,total_electricity,total_hhd
0,2458,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2467,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2469,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2491,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2493,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
545,4846,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
546,4848,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
547,4849,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
548,4850,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
sd_electricity['avg_electricity'] = sd_electricity['total_electricity']/sd_electricity['total_hhd']

In [ ]:
sd_electricity

,sub_district_code,rural_hhd,rural_electricity,urban_hhd,urban_electricity,urban_tele,urban_hhd_pipe,urban_no_sanitation,total_electricity,total_hhd,avg_electricity
0,2458,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
1,2467,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
2,2469,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
3,2491,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
4,2493,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...
545,4846,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
546,4848,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
547,4849,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
548,4850,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN


## availability_of_telephone_services

In [ ]:
telephone_df = vul_df.copy()

In [ ]:
telephone_df['rural_tele'] = vul_df['rural_hhd']*vul_df['availability_of_telephone_services']

In [ ]:
telephone_df = telephone_df[['sub_district_code', 'rural_hhd', 'rural_tele']]

In [ ]:
telephone_df = telephone_df.groupby('sub_district_code')[['rural_hhd','rural_tele']].sum().reset_index()

In [ ]:
#dependent on urban_shapes
#rural_hhd['sub_district_code'] = rural_hhd['sub_district_code'].astype(str)
sd_tele = telephone_df.merge(urban_hhd[['sub_district_code', 'urban_hhd','urban_tele']], on='sub_district_code', how='outer')

In [ ]:
sd_tele = sd_tele.fillna(0)

In [ ]:
sd_tele['total_tele'] = sd_tele['rural_tele'] + sd_tele['urban_tele']

In [ ]:
sd_tele['total_hhd'] = sd_tele['rural_hhd'] + sd_tele['urban_hhd']

In [ ]:
sd_tele['avg_tele'] = round(sd_tele['total_tele']/sd_tele['total_hhd'])

In [ ]:
sd_tele

,sub_district_code,rural_hhd,rural_tele,urban_hhd,urban_tele,total_tele,total_hhd,avg_tele
0,2458,0.0,0.0,0.0,0.0,0.0,0.0,NaN
1,2467,0.0,0.0,0.0,0.0,0.0,0.0,NaN
2,2469,0.0,0.0,0.0,0.0,0.0,0.0,NaN
3,2491,0.0,0.0,0.0,0.0,0.0,0.0,NaN
4,2493,0.0,0.0,0.0,0.0,0.0,0.0,NaN
...,...,...,...,...,...,...,...,...
545,4846,0.0,0.0,0.0,0.0,0.0,0.0,NaN
546,4848,0.0,0.0,0.0,0.0,0.0,0.0,NaN
547,4849,0.0,0.0,0.0,0.0,0.0,0.0,NaN
548,4850,0.0,0.0,0.0,0.0,0.0,0.0,NaN


## total_hhd_having_piped_water_connection

In [ ]:
pipe_df = vul_df.copy()

In [ ]:
pipe_df = pipe_df.groupby('sub_district_code')[['total_hhd_having_piped_water_connection']].sum().reset_index()

In [ ]:
pipe_df = pipe_df.merge(urban_hhd[['sub_district_code','urban_hhd_pipe']], on='sub_district_code', how='outer')

In [ ]:
pipe_df = pipe_df.fillna(0)

In [ ]:
pipe_df['sd_piped_hhds'] = pipe_df['urban_hhd_pipe'] + pipe_df['total_hhd_having_piped_water_connection']

In [ ]:
pipe_df = pipe_df.merge(total_hhd[['sub_district_code','total_hhd']], on='sub_district_code')

In [ ]:
pipe_df['sd_piped_hhds_pct'] = 100*pipe_df['sd_piped_hhds']/pipe_df['total_hhd']

In [ ]:
pipe_df

,sub_district_code,total_hhd_having_piped_water_connection,urban_hhd_pipe,sd_piped_hhds,total_hhd,rc_piped_hhds_pct,sd_piped_hhds_pct
0,2458,0.0,0.0,0.0,0.0,NaN,NaN
1,2467,0.0,0.0,0.0,0.0,NaN,NaN
2,2469,0.0,0.0,0.0,0.0,NaN,NaN
3,2491,0.0,0.0,0.0,0.0,NaN,NaN
4,2493,0.0,0.0,0.0,0.0,NaN,NaN
...,...,...,...,...,...,...,...
545,4846,0.0,0.0,0.0,0.0,NaN,NaN
546,4848,0.0,0.0,0.0,0.0,NaN,NaN
547,4849,0.0,0.0,0.0,0.0,NaN,NaN
548,4850,0.0,0.0,0.0,0.0,NaN,NaN


## total_hhd_not_having_sanitary_latrines

In [ ]:
nosanitation_df = vul_df.copy()

In [ ]:
nosanitation_df = nosanitation_df.groupby('sub_district_code')[['total_hhd_not_having_sanitary_latrines']].sum().reset_index()

In [ ]:
nosanitation_df = nosanitation_df.merge(urban_hhd[['sub_district_code','urban_no_sanitation']], on='sub_district_code', how='outer')

In [ ]:
nosanitation_df = nosanitation_df.fillna(0)

In [ ]:
nosanitation_df['sd_nosanitation_hhds'] = nosanitation_df['urban_no_sanitation'] + nosanitation_df['total_hhd_not_having_sanitary_latrines']

In [ ]:
nosanitation_df = nosanitation_df.merge(total_hhd[['sub_district_code','total_hhd']], on='sub_district_code')

In [ ]:
nosanitation_df['sd_nosanitation_hhds_pct'] = 100*nosanitation_df['sd_nosanitation_hhds']/nosanitation_df['total_hhd']

In [ ]:
nosanitation_df

,sub_district_code,total_hhd_not_having_sanitary_latrines,urban_no_sanitation,sd_nosanitation_hhds,total_hhd,sd_nosanitation_hhds_pct
0,2458,0.0,0.0,0.0,0.0,NaN
1,2467,0.0,0.0,0.0,0.0,NaN
2,2469,0.0,0.0,0.0,0.0,NaN
3,2491,0.0,0.0,0.0,0.0,NaN
4,2493,0.0,0.0,0.0,0.0,NaN
...,...,...,...,...,...,...
545,4846,0.0,0.0,0.0,0.0,NaN
546,4848,0.0,0.0,0.0,0.0,NaN
547,4849,0.0,0.0,0.0,0.0,NaN
548,4850,0.0,0.0,0.0,0.0,NaN


# ANTYODAYA MASTER

In [ ]:
antyodaya_master_df = net_sown_area_in_hac_sd.reset_index()

In [ ]:
antyodaya_master_df = antyodaya_master_df.merge(sd_electricity[['sub_district_code', 'avg_electricity']], how='outer')

In [ ]:
antyodaya_master_df = antyodaya_master_df.merge(sd_tele[['sub_district_code', 'avg_tele']])

In [ ]:
antyodaya_master_df = antyodaya_master_df.merge(pipe_df[['sub_district_code', 'sd_piped_hhds_pct']])

In [ ]:
antyodaya_master_df = antyodaya_master_df.merge(nosanitation_df[['sub_district_code','sd_nosanitation_hhds_pct']])


In [ ]:
antyodaya_master_df

,sub_district_code,net_sown_area_in_hac,avg_electricity,avg_tele,sd_piped_hhds_pct,sd_nosanitation_hhds_pct
0,2458,NaN,NaN,NaN,NaN,NaN
1,2467,NaN,NaN,NaN,NaN,NaN
2,2469,NaN,NaN,NaN,NaN,NaN
3,2491,NaN,NaN,NaN,NaN,NaN
4,2493,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
545,4846,NaN,NaN,NaN,NaN,NaN
546,4848,NaN,NaN,NaN,NaN,NaN
547,4849,NaN,NaN,NaN,NaN,NaN
548,4850,NaN,NaN,NaN,NaN,NaN


In [ ]:
gdf_polygons

,OBJECTID,stcode11,dtcode11,sdtcode11,Shape_Length,Shape_Area,stname,dtname,sdtname,Subdt_LGD,Dist_LGD,State_LGD,geometry
0,2773,21,370,02755,175809.421683,6.516630e+08,ODISHA,Bargarh,Paikamal,2755,347,21,"POLYGON ((82.90633 21.04445, 82.90599 21.04534..."
1,2774,21,370,02756,192057.189614,7.626096e+08,ODISHA,Bargarh,Jharbandha,2756,347,21,"POLYGON ((82.96518 21.19248, 82.96504 21.19318..."
2,2775,21,370,02757,133403.031651,3.456675e+08,ODISHA,Bargarh,Padmapur,2757,347,21,"POLYGON ((83.16820 21.13856, 83.16775 21.13887..."
3,2776,21,370,02758,113949.812620,2.672039e+08,ODISHA,Bargarh,BurdenP.S.,2758,347,21,"POLYGON ((83.12346 20.98452, 83.12192 20.98650..."
4,2777,21,370,02759,135841.048349,3.752092e+08,ODISHA,Bargarh,Gaisilet,2759,347,21,"POLYGON ((83.28376 21.04885, 83.28268 21.04889..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
473,3246,21,399,03226,149569.992418,5.100200e+08,ODISHA,Malkangiri,Orkel,3226,364,21,"POLYGON ((82.16914 18.48265, 82.16793 18.48307..."
474,3247,21,399,03227,269711.225643,9.885963e+08,ODISHA,Malkangiri,Kalimela,3227,364,21,"POLYGON ((81.72320 18.29129, 81.72195 18.29135..."
475,3248,21,399,03228,129218.450510,4.291734e+08,ODISHA,Malkangiri,Podia,3228,364,21,"POLYGON ((81.66331 18.32195, 81.66271 18.32223..."
476,3249,21,399,03229,130042.312962,3.676498e+08,ODISHA,Malkangiri,M.V. 79,3229,364,21,"POLYGON ((81.64158 18.10071, 81.64075 18.10174..."


In [ ]:
antyodaya_master_df.head()

,sub_district_code,net_sown_area_in_hac,avg_electricity,avg_tele,sd_piped_hhds_pct,sd_nosanitation_hhds_pct
0,2458,NaN,NaN,NaN,NaN,NaN
1,2467,NaN,NaN,NaN,NaN,NaN
2,2469,NaN,NaN,NaN,NaN,NaN
3,2491,NaN,NaN,NaN,NaN,NaN
4,2493,NaN,NaN,NaN,NaN,NaN


In [ ]:
#gdf_polygons = gdf_polygons.rename(columns = {'SDTCODE11':'sub_district_code'}, inplace = True)
antyodaya_master_df = pd.merge(gdf_polygons_renamed,antyodaya_master_df,on='sub_district_code', how='outer')

In [ ]:
antyodaya_master_df[antyodaya_master_df.avg_electricity.isnull()]

,OBJECTID,stcode11,dtcode11,sub_district_code,Shape_Length,Shape_Area,stname,dtname,sdtname,Subdt_LGD,Dist_LGD,State_LGD,geometry,net_sown_area_in_hac,avg_electricity,avg_tele,sd_piped_hhds_pct,sd_nosanitation_hhds_pct
0,NaN,NaN,NaN,2458,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,2467,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,2469,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,2491,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,2493,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
549,NaN,NaN,NaN,4846,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN
550,NaN,NaN,NaN,4848,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN
551,NaN,NaN,NaN,4849,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN
552,NaN,NaN,NaN,4850,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN


In [ ]:
antyodaya_master_df = antyodaya_master_df.merge(total_hhd[['sub_district_code', 'total_hhd']], on='sub_district_code')
antyodaya_master_df['total_hhd'] = antyodaya_master_df['total_hhd'].apply(lambda x: round(x,0))

In [ ]:
antyodaya_master_df.to_csv(r'D:\CivicDataLabs_IDS-DRR\IDS-DRR_Github\flood-data-ecosystem-Odisha\Sources\ANTYODAYA\data\antyodaya_variables.csv', index=False)

In [ ]:
#original code for reference
'''antyodaya_master_df.drop(['geometry',
                           'district_1', 'revenue_cr', 'HQ', 'area', 
                           'are_new'],axis=1).to_csv('/home/krishna/IDS-DRR-Data-Pipeline/Sources/ANTYODAYA/data/variables/antyodaya/antyodaya_variables.csv', index=False)
'''

"antyodaya_master_df.drop(['geometry',\n                           'district_1', 'revenue_cr', 'HQ', 'area', \n                           'are_new'],axis=1).to_csv('/home/krishna/IDS-DRR-Data-Pipeline/Sources/ANTYODAYA/data/variables/antyodaya/antyodaya_variables.csv', index=False)\n"

In [ ]:
antyodaya_master_df

,OBJECTID,stcode11,dtcode11,sub_district_code,Shape_Length,Shape_Area,stname,dtname,sdtname,Subdt_LGD,Dist_LGD,State_LGD,geometry,net_sown_area_in_hac,avg_electricity,avg_tele,sd_piped_hhds_pct,sd_nosanitation_hhds_pct,total_hhd
0,NaN,NaN,NaN,2458,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,0.0
1,NaN,NaN,NaN,2467,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,0.0
2,NaN,NaN,NaN,2469,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,0.0
3,NaN,NaN,NaN,2491,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,0.0
4,NaN,NaN,NaN,2493,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
547,NaN,NaN,NaN,4846,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,0.0
548,NaN,NaN,NaN,4848,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,0.0
549,NaN,NaN,NaN,4849,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,0.0
550,NaN,NaN,NaN,4850,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,0.0
